<a href="https://colab.research.google.com/github/mialdrid/Python/blob/master/MAldridge_IA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt') # tokenizer
nltk.download('wordnet') # lemmatizer
nltk.download('stopwords') # used to handle words like a, an, the
nltk.download('averaged_perceptron_tagger') # Part of Speech

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
np.random.seed(2018)

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import *
from sklearn.metrics.pairwise import *
from sklearn.cluster import AgglomerativeClustering

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
#Read in csv file
df = pd.read_csv("https://raw.githubusercontent.com/AnoVando/MSIS/master/IA3.csv")
#head() to verify that data read in properly
df.head()

,id,review,label
0,1,About the shop: There is a restaurant in Soi L...,restaurant
1,2,About the shop: Through this store for about t...,restaurant
2,3,Roast Coffee &amp; Eatery is a restaurant loca...,restaurant
3,4,Eat from the children. The shop is opposite. P...,restaurant
4,5,The Ak 1 shop at another branch tastes the sam...,restaurant


In [3]:
#Check for any null values
df.isnull().any()

id        False
review    False
label     False
dtype: bool

In [0]:
#Create list containing the review data
review_data=df["review"].tolist()

In [5]:
#Step 1: Tokenize
token_review =[]
for i in review_data:
  token = nltk.word_tokenize(i)
  token_review.append(token)

print(token_review)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
#1 term-doc matrix and lemmatize
lemmatize_review = []
lemmatizer = nltk.stem.WordNetLemmatizer()
for i in token_review:
  lemmatized_review = [lemmatizer.lemmatize(x) for x in i if x.isalpha()]
  lemmatize_review.append(lemmatized_review)
print(lemmatize_review)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
#remove stopwords
from nltk.corpus import stopwords
stopWordEnglish = stopwords.words('english')
stopword_list=['I','l','lt','gt','quot','wa','ha','Ourselves','Hers','Between','Yourself','But','Again','There','About','Once',
               'During','Out','Very','Having','With','They','Own','an','Be','Some','For','Do', 'Its', 'Yours', 'Such', 'Into',
               'Of', 'Most', 'Itself', 'Other', 'Off', 'Is', 'S', 'Am', 'Or', 'Who', 'As', 'From', 'Him', 'Each', 'The', 'Themselves',
               'Until', 'Below', 'Are', 'We', 'These', 'Your', 'His', 'Through', 'Don', 'Nor', 'Me', 'Were', 'Her', 'More', 'Himself',
               'This', 'Down', 'Should', 'Our', 'Their', 'While', 'Above', 'Both', 'Up', 'To', 'Ours', 'Had', 'She', 'All', 'No', 'When',
               'At', 'Any', 'Before', 'Them', 'Same', 'And', 'Been', 'Have', 'In', 'Will', 'On', 'Does', 'Yourselves', 'Then', 'That', 'Because',
               'What', 'Over', 'Why', 'So', 'Can', 'Did', 'Not', 'Now', 'Under', 'He', 'You', 'Herself', 'Has', 'Just', 'Where', 'Too', 'Only',
               'Myself', 'which', 'those', 'After', 'Few', 'Whom', 'T', 'Being', 'If', 'Theirs', 'My', 'Against', 'A', 'By', 'Doing', 'It', 'How',
               'Further', 'Was', 'Here', 'Than']
stopWordEnglish.extend(stopword_list)
stop_words = []

for stop in lemmatize_review:
  stop_words_removed = [x for x in stop if x not in stopWordEnglish]
  stop_words.append(stop_words_removed)

In [8]:
#Create a tf-id vector in order to run the lda model
from sklearn.feature_extraction.text import CountVectorizer

input2 = [" ".join(x) for x in stop_words]

count = CountVectorizer(ngram_range=(2, 2), min_df=5)
count.fit(input2)
v2 = count.transform(input2)
terms = count.get_feature_names()
print(terms)
print(count.vocabulary_)
print('\n')
print(v2.toarray())


['acting skill', 'action film', 'action movie', 'actor also', 'add little', 'add lot', 'addition also', 'addition film', 'advantage disadvantage', 'air room', 'almost every', 'along way', 'also available', 'also become', 'also called', 'also caused', 'also gave', 'also good', 'also know', 'also known', 'also let', 'also like', 'also line', 'also little', 'also made', 'also make', 'also many', 'also need', 'also said', 'also see', 'also show', 'also simple', 'also think', 'also true', 'also want', 'although film', 'although know', 'although many', 'although plot', 'although still', 'although story', 'always feel', 'always felt', 'always make', 'always people', 'always think', 'always thought', 'american drama', 'american movie', 'ancient time', 'animated film', 'another menu', 'another place', 'another point', 'anything else', 'around world', 'art film', 'atmosphere good', 'atmosphere nice', 'atmosphere restaurant', 'atmosphere shop', 'audience also', 'audience like', 'audience watching

In [9]:
#2 create an lda model to extract topics
#4 define the top five terms across each topic
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=6).fit(v2)

for i,topic in enumerate(lda.components_):
    print(f'Top 10 words for topic #{i}:')
    print([terms[i] for i in topic.argsort()[-5:]])
    print('\n')
first_term=lda.transform(v2[0])
second_term=lda.transform(v2[1])
third_term=lda.transform(v2[2])
fouth_term=lda.transform(v2[3])
fifth_term=lda.transform(v2[2])

Top 10 words for topic #0:
['many people', 'united states', 'tv series', 'war film', 'tv drama']


Top 10 words for topic #1:
['united states', 'golden shelling', 'good friend', 'year old', 'many people']


Top 10 words for topic #2:
['martial art', 'make people', 'kung fu', 'hong kong', 'box office']


Top 10 words for topic #3:
['one day', 'many people', 'blind date', 'river lake', 'every day']


Top 10 words for topic #4:
['taste good', 'good taste', 'like eat', 'green tea', 'ice cream']


Top 10 words for topic #5:
['step step', 'film also', 'love love', 'horror film', 'elf hostel']




Topic 0 seems to focus on different types of tv programs

Topic 1 seems to focus on demographics across the united state

Topic 2 steems to focus on martial-arts focused movies

Topic 3 seems to focus on blind dating

Topic 4 seems to focus on food and drink preferences and tastes

Topic 5 seems to focus on film and movie preferences

In [10]:
#Prepare data to report topic distribution
v_food = v2[1:10]
v_movie = v2[501:510]
print(v_food)
print(v_movie)

  (0, 64)	1
  (0, 125)	1
  (0, 155)	2
  (0, 177)	1
  (0, 183)	1
  (0, 445)	1
  (0, 463)	1
  (0, 467)	1
  (0, 522)	1
  (0, 545)	1
  (0, 547)	1
  (0, 699)	1
  (0, 725)	1
  (0, 919)	1
  (0, 1038)	1
  (0, 1119)	1
  (0, 1123)	1
  (0, 1184)	1
  (0, 1245)	1
  (0, 1322)	1
  (0, 1359)	1
  (0, 1370)	1
  (0, 1374)	1
  (0, 1401)	1
  (0, 1454)	1
  :	:
  (5, 821)	1
  (5, 992)	1
  (5, 1214)	1
  (6, 51)	1
  (6, 57)	1
  (6, 222)	2
  (6, 252)	1
  (6, 461)	1
  (6, 904)	1
  (6, 957)	1
  (6, 1052)	1
  (6, 1059)	1
  (6, 1154)	1
  (6, 1193)	1
  (6, 1290)	1
  (6, 1305)	2
  (6, 1323)	1
  (6, 1401)	1
  (6, 1412)	2
  (6, 1424)	2
  (7, 57)	1
  (7, 516)	1
  (7, 547)	1
  (8, 261)	1
  (8, 629)	1
  (0, 88)	1
  (0, 244)	1
  (0, 598)	1
  (0, 627)	1
  (0, 727)	2
  (0, 798)	1
  (0, 821)	1
  (0, 825)	2
  (0, 888)	1
  (0, 1003)	1
  (0, 1010)	2
  (0, 1011)	1
  (0, 1347)	1
  (0, 1394)	1
  (0, 1399)	1
  (0, 1474)	1
  (0, 1531)	2
  (1, 112)	1
  (1, 144)	1
  (1, 317)	1
  (1, 338)	1
  (1, 369)	1
  (1, 406)	1
  (1, 509)	1
  (1, 5

In [11]:
#3 Report topic distribution - Food Review
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=6).fit(v_food)

for i,topic in enumerate(lda.components_):
    print(f'Top 10 words for topic #{i}:')
    print([terms[i] for i in topic.argsort()[-5:]])
    print('\n')

Top 10 words for topic #0:
['bottle baht', 'time try', 'good service', 'value money', 'chicken wing']


Top 10 words for topic #1:
['eat lot', 'people eat', 'people like', 'eat restaurant', 'price baht']


Top 10 words for topic #2:
['people eat', 'people like', 'opposite side', 'shop opposite', 'food eat']


Top 10 words for topic #3:
['atmosphere good', 'try try', 'delicious delicious', 'tomato sauce', 'sweet sour']


Top 10 words for topic #4:
['park car', 'good atmosphere', 'atmosphere good', 'doe know', 'know restaurant']


Top 10 words for topic #5:
['sauce sweet', 'sweet taste', 'soup sweet', 'sweet delicious', 'per person']




In [12]:
#3 Report topic distribution - Movie review
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=6).fit(v_movie)

for i,topic in enumerate(lda.components_):
    print(f'Top 10 words for topic #{i}:')
    print([terms[i] for i in topic.argsort()[-5:]])
    print('\n')

Top 10 words for topic #0:
['united states', 'passed away', 'time passed', 'film television', 'box office']


Top 10 words for topic #1:
['long time', 'people think', 'world view', 'many time', 'many people']


Top 10 words for topic #2:
['people also', 'make difficult', 'middle school', 'film good', 'mobile phone']


Top 10 words for topic #3:
['make people', 'two people', 'three year', 'men woman', 'blind date']


Top 10 words for topic #4:
['much like', 'every day', 'part life', 'nothing wrong', 'one time']


Top 10 words for topic #5:
['film market', 'still many', 'bit like', 'tell story', 'domestic film']




In [0]:
print(review_data[0])

The first review describes the decor, food quality, service and overall dining experience of the restaurant, Soi Langsuan, which is located inside of the Luxx hotel

In [0]:
print(review_data[500])

The last review is about a girl reflecting on a specific relationship and how she sees that impacting her life in the future. She asks specific questions about how the first meaningful relationship someone experiences defines their approach to relationships further in life